In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


In [ ]:
# # Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="gemma2-9b-it",
    temperature=0.7
)

# llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o")


# 모델 성능을 높이는 프롬프트 기법 
- 모델에게 정확한 지시를 제공하고, 원하는 출력을 얻기 위해 입력을 최적화하는 기술

### 1) 구체적이고 명확하게 사용자의 지시를 제공합니다.
- 프롬프트는 모델이 이해하기 쉽게 명확하고 간결해야 합니다.  
- 불필요한 정보를 줄이고, 핵심 요구 사항에 집중해야 합니다.
- 원하는 출력이 무엇인지 모델에게 정확하게 알려주어야 합니다. 

In [ ]:
# 질문 (Question)

# llm.invoke("GPT-3.5 모델의 파라미터 개수는 몇개인가요? 답변은 아라비아 숫자만을 사용해서 답변해주세요. ")
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 {system_input} 전문가입니다.") , 
     ("user", "How many parameters does the {user_input} model have? Please display the number in the format 1,000,000") ]
)

# answer using only numerals.
# display the number in the format 1,000,000
prompt_text = prompt.format(system_input="AI", user_input="mistral-saba-24b") #gpt-3.5-turbo" / gpt-4o / mistral-saba-24b / llama-4-scout-17b-16e-instruct
response = llm.invoke(prompt_text)
print(response.content)

In [ ]:
# 컨텍스트 제공 (Context)

response = llm.invoke(
    """다음 제시된 뉴스를 기반으로 질문에 답변하세요:
    뉴스: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
          이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점에 도전하고, 
          세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
    
    질문: AI 반도체 시장에서 지배적인 위치를 차지하고 있는 회사는 어디인가요? 회사이름만 출력해주세요.
    답변:
    """)
print(response.content)

### 2) 참고할 수 있는 예시를 제공합니다.
- 원하는 출력 형식이나 스타일을 모델에게 보여주기 위해 예시를 사용할 수 있습니다. 
- 이는 모델이 출력의 방향을 잡는데 도움이 됩니다.

In [ ]:
# 예시 없음 (zero-shot)

response = llm.invoke(
    """다음 제시된 뉴스에서 3개의 키워드를 추출하세요:
    뉴스: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
          이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점에에 도전하고, 
          세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
    
    키워드: 
    """)
print(response.content)

In [ ]:
# 1개의 예시를 제공 (one-shot)

response = llm.invoke(
    """다음 예시와 같이 제시된 뉴스에서 3개의 키워드를 추출하세요:
    <예시>
    뉴스: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
          이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
          세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
    
    키워드: 삼성전자, 인공지능, 엔비디아
    </예시>

    AI의 영향을 가장 크게 받은 구글 제품은 바로 구글 검색입니다. 
    현재 10억 명의 이용자가 구글의 AI 개요(AI Overviews) 기능을 통해 완전히 새로운 유형의 질문을 할 수 있게 됐으며, 이는 가장 인기 있는 검색 기능 중 하나가 됐습니다. 
    구글은 다음 단계로, 제미나이 2.0의 고급 추론 기능을 AI 개요에 적용해 고급 수학 방정식, 멀티모달 쿼리 및 코딩 등 더 복잡한 질문을 처리할 수 있도록 개선할 예정입니다. 
    구글은 이번 주에 제한된 범위의 테스트를 시작했으며 내년 초에 더 광범위하게 출시할 예정입니다. 
    또한 내년에는 AI 개요 기능을 더 많은 국가와 언어로 확대해 선보일 계획입니다.
    
    키워드:
    """)
print(response.content)

In [ ]:
# 여러 개의 예시를 제공 (few-shot)
response = llm.invoke(
    """다음 예시들과 같이 제시된 뉴스에서 각각 3개의 키워드를 추출하세요:
    <예시1>
    뉴스: 삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 
          이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 
          세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.
    키워드: 삼성전자, 인공지능, 엔비디아
    </예시1>

    <예시2>
    뉴스: 세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 
          전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.
    키워드: 세계보건기구 | 건강위기 | 국제 
    </예시2>

    뉴스: 제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI) 에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
         모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
         내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
      
     키워드:
    """
)
# gpt-4o
# 키워드: 제미나이 2.0, 구글 AI 스튜디오, 버텍스 AI 
print(response.content)

### 3) 순차적인 프롬프트(Chain of Thought)를 적용합니다.
복잡한 문제를 해결할 때, 단계별로 문제를 분해하여 모델이 각 단계를 순차적으로 해결하도록 유도합니다.

In [ ]:
# zero-shot 예시
llm.invoke(
    """
    Question: 학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
              5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
              6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
              과학 동아리에는 몇 명의 학생이 있나요?
    Answer:
    """
)

In [ ]:
# few-shot 예시
llm.invoke(
    """
    Question: 학교에서 300명의 학생이 있습니다. 이 중 40%는 4학년입니다. 4학년 학생들 중 절반은 축구 팀에 있고, 나머지 절반은 음악 클럽에 있습니다. 
              축구 팀에 몇 명의 학생이 있나요?
    Answer: 
    1. 첫번째 단계: 학교에는 총 300명의 학생이 있으며, 이 중 40%가 4학년입니다. 따라서, 4학년 학생 수는 전체 학생 수의 40%에 해당합니다."
    2. 두번째 단계: 4학년 학생들 중 절반은 축구 팀에 있습니다. 따라서, 축구 팀에 있는 4학년 학생 수는 4학년 학생 수의 절반에 해당합니다."
    3. 세번째 단계: 첫 번째 단계에서 구한 4학년 학생 수의 절반을 두 번째 단계의 계산으로 구합니다.
    따라서, 축구 팀에 있는 4학년 학생 수는 300 * 40% * 50% = 60명입니다.

    Question: 학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
              5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
              6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
              과학 동아리에는 몇 명의 학생이 있나요?
    Answer:
    """
)


In [ ]:
# think step by step
llm.invoke(
    """
    Question: 학교에서 500명의 학생이 있습니다. 이 중 30%는 5학년이고, 20%는 6학년 학생입니다. 
              5학년 학생들 중 60%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다. 
              6학년 학생들 중 70%는 수학 동아리에 있고, 나머지는 과학 동아리에 있습니다.
              과학 동아리에는 몇 명의 학생이 있나요?
              Let's think step by step.
    Answer:
    """
)